In [1]:
import matplotlib.pyplot as plt
from pathlib import Path
import re

In [2]:
# Regular expression to match the output of the simulation
LOSS_PER_IT = re.compile(r"[0-9]*\/[0-9]*.*iteration\s[0-9]*\s:\sloss\s:\s[0-9]*\.[0-9]*,\sloss_ce:\s[0-9]*\.[0-9]*\niteration\s[0-9]*\s:\sloss\s:\s[0-9]*\.[0-9]*,\sloss_ce:\s[0-9]*\.[0-9]*")
TEST_PER_IM = re.compile(r"idx\s[1-9]\scase\saugmented_[0-9]*\smean_dice\s[0-9\.]mean_hd95\s[0-9\.]")
MEAN_TEST   = re.compile(r"Testing\sperformance\sin\sbest\sval\smodel:\smean_dice\s:\s[0-9]*\smean_hd95\s:\s[0-9]*")

In [3]:
# File where the run outputs have been written to
path = Path("./outputs/Output.txt")
data = None
with open(path, "rb") as f:
	data = f.read().decode("utf-8", "replace")

In [4]:
# This dictionary will contain all the parsed data
data_per_epoch = {}

In [5]:
# First loss and loss_ce per iteration for all the different max_epocs run
for match in LOSS_PER_IT.finditer(data):
	matchstr = match.group(0)
	# Get the epoch number
	current_epoch, max_epochs = matchstr.split(" ")[0].split("/")
	# If the max_epochs is not in the dictionary, add it
	if max_epochs not in data_per_epoch:
		data_per_epoch[max_epochs] = []
	# Get the loss and loss_ce
	loss, loss_ce = matchstr.split("loss : ")[1].split(", loss_ce: ")
	# Add the loss and loss_ce to the dictionary
	data_per_epoch[max_epochs].append((float(loss), float(loss_ce)))

# Get the mean dice and mean hd95 for all the different max_epocs run
for match in MEAN_TEST.finditer(data):
	matchstr = match.group(0)
	# Get the epoch number
	current_epoch, max_epochs = matchstr.split("mean_dice : ")[1].split(" mean_hd95 ")
	# If the max_epochs is not in the dictionary, add it
	if max_epochs not in data_per_epoch:
		data_per_epoch[max_epochs] = []
	# Get the mean dice and mean hd95
	mean_dice, mean_hd95 = matchstr.split("mean_dice : ")[1].split(" mean_hd95 : ")
	# Add the mean dice and mean hd95 to the dictionary
	data_per_epoch[max_epochs].append((mean_dice, mean_hd95))

# Plot the loss and loss_ce for each max_epochs
for max_epochs in data_per_epoch:
	# Get the loss and loss_ce for each epoch
	loss, loss_ce = zip(*data_per_epoch[max_epochs][0::2])
	# Plot the loss and loss_ce
	plt.plot(loss, label="loss")
	plt.plot(loss_ce, label="loss_ce")
	plt.xlabel("Epoch")
	plt.ylabel("Loss")
	plt.legend()
	plt.title("Loss for max_epochs = {}".format(max_epochs))
	plt.show()

	# Get the mean dice and mean hd95 for each epoch
	mean_dice, mean_hd95 = zip(*data_per_epoch[max_epochs][1::2])
	# Plot the mean dice and mean hd95
	plt.plot(mean_dice, label="mean_dice")
	plt.plot(mean_hd95, label="mean_hd95")
	plt.xlabel("Epoch")
	plt.ylabel("Mean")
	plt.legend()
	plt.title("Mean for max_epochs = {}".format(max_epochs))
	plt.show()

ValueError: not enough values to unpack (expected 2, got 1)